In [5]:
import sqlite3

# Verbindung zur SQLite-Datenbank herstellen
conn = sqlite3.connect('sentiment.db')
cursor = conn.cursor()

# Abfrage aller relevanten Daten
query = "SELECT id, text_id, sentence_text, sequence, finbert_result, finbert_score, dffnsa_result, dffnsa_score, fsa_result, fsa_score FROM sentences"
cursor.execute(query)
rows = cursor.fetchall()

# rows = rows[:30]

# Datenbankverbindung schließen
conn.close()

import pandas as pd
from collections import defaultdict

# Daten in einen DataFrame laden
df = pd.DataFrame(rows, columns=['id', 'text_id', 'sentence_text', 'sequence', 'finbert_result', 'finbert_score', 'dffnsa_result', 'dffnsa_score', 'fsa_result', 'fsa_score'])

# Definition der gewichteten Mehrheitsentscheidungsfunktion
def calculate_final_sentiment(row):
    # Sammle die Vorhersagen in einem Dictionary, wobei der Schlüssel das Sentiment ist
    # und der Wert eine Liste der Konfidenzwerte
    sentiment_votes = defaultdict(list)
    
    # Füge die Konfidenzwerte für jedes Sentiment hinzu
    for model in ['finbert', 'dffnsa', 'fsa']:
        sentiment = row[f'{model}_result']
        score = row[f'{model}_score']
        sentiment_votes[sentiment].append(score)
    
    # Berechne den durchschnittlichen Konfidenzwert für jedes Sentiment
    average_confidence = {sentiment: sum(scores)/len(scores) for sentiment, scores in sentiment_votes.items()}
    
    # Bestimme die Mehrheit basierend auf der Anzahl der Stimmen und der durchschnittlichen Konfidenz
    majority_sentiment = max(sentiment_votes, key=lambda k: (len(sentiment_votes[k]), average_confidence[k]))
    
    # Überprüfe, ob ein anderes Sentiment eine deutlich höhere Konfidenz hat
    highest_confidence_sentiment = max(average_confidence, key=average_confidence.get)
    if average_confidence[highest_confidence_sentiment] > max(0.9, average_confidence[majority_sentiment] + 0.1):
        # Wenn ein anderes Sentiment eine signifikant höhere durchschnittliche Konfidenz hat,
        # markiere diesen Satz für manuelle Überprüfung.
        return 'manual'
    
    return majority_sentiment

# Anwenden der Funktion auf jede Zeile des DataFrame
df['final_sentiment'] = df.apply(calculate_final_sentiment, axis=1)

print(df.head(10))

   id  text_id                                      sentence_text  sequence  \
0   1        1  Chinese-owned companies are aggressively expan...         0   
1   2        1  Chinese cobalt producers have seemed unfazed b...         1   
2   3        1  China's CMOC Group, which boosted its cobalt o...         2   
3   4        1  CMOC is due to lift its market share of the gl...         3   
4   5        1  Its Kisanfu mine in Democratic Republic of Con...         4   
5   6        1  The group is able to operate at low costs, lik...         5   
6   7        1  CMOC is listed in Hong Kong, but according to ...         6   
7   8        1  “Is CMOC trying to flood the cobalt market in ...         7   
8   9        1  When contacted for comment about the impact of...         8   
9  10        1  "The growth of cobalt supply has to some exten...         9   

  finbert_result  finbert_score dffnsa_result  dffnsa_score fsa_result  \
0        bullish          0.909       bullish         1.

In [6]:
# Verbindung zur SQLite-Datenbank wiederherstellen
conn = sqlite3.connect('sentiment.db')
cursor = conn.cursor()

# Update-Abfrage für jedes finale Sentiment
for index, row in df.iterrows():
    cursor.execute('UPDATE sentences SET final_sentiment = ? WHERE id = ?', (row['final_sentiment'], row['id']))

# Änderungen speichern und Datenbankverbindung schließen
conn.commit()
conn.close()
